# Adding Adapter Support to Qwen 2.5 via Plugin Interface

This notebook demonstrates how to add adapter support to a custom or not pre-supported model with the _Adapters_ library's **[plugin interface](https://docs.adapterhub.ml/plugin_interface.html)**. Specifically, we'll be writing a plugin interface for the **Qwen 2.5** model and then train an adapter for mathematical reasoning.

## 1. Installing Required Libraries

First, let's install the necessary libraries if you haven't already.

In [ ]:
# Install the adapters library and transformers
!pip install -q adapters transformers datasets trl

## 2. Understanding the Model Architecture

Before creating our plugin interface, let's understand the basic structure of Qwen 2.5:

- Like most Transformer language models, it consists of an embedding layer followed by a series of decoder layers
- Each layer contains a self-attention block and an MLP block
- The self-attention block includes query, key, value, and output projections
- The MLP block includes multiple linear projections
- Qwen applies layer norms *before* the self-attention and MLP blocks

To create an adapter interface, we need to map these components to the appropriate adapter hooks.

## 3. Creating the Plugin Interface

Now we'll create a plugin interface for Qwen 2.5 that maps the model's architecture to the adapter framework.

‼️ The interface below for Qwen 2 and Qwen 2.5 already comes pre-supported in _Adapters_, so you could skip this section entirely! It's merely to showcase how you could define interfaces for your own custom models!

You can find a list of all pre-supported models here: https://docs.adapterhub.ml/model_overview.html.

In [ ]:
import adapters
from adapters import AdapterModelInterface
from transformers import AutoModelForCausalLM

plugin_interface = AdapterModelInterface(
    # Specify which adapter methods to enable
    adapter_methods=["lora", "reft", "bottleneck"],
    
    # Map the model's components to the adapter interface
    model_embeddings="embed_tokens",      # Embedding layer
    model_layers="layers",                # Transformer layers
    layer_self_attn="self_attn",          # Self-attention module in each layer
    layer_cross_attn=None,                # Qwen doesn't have cross-attention
    
    # Projection matrices within the attention module
    attn_k_proj="k_proj",                 # Key projection
    attn_q_proj="q_proj",                 # Query projection
    attn_v_proj="v_proj",                 # Value projection
    attn_o_proj="o_proj",                 # Output projection
    
    # MLP projections
    layer_intermediate_proj="mlp.up_proj",  # Up projection in MLP
    layer_output_proj="mlp.down_proj",      # Downward projection in MLP

    layer_pre_self_attn="input_layernorm",  # Hook directly before self-attention
    layer_pre_ffn="post_attention_layernorm",  # Hook directly before MLP
    # Qwen applies layer norms before attention and MLP, so no need to add them here
    layer_ln_1=None,
    layer_ln_2=None,
)

Each parameter in the interface maps to specific module names in the model's architecture, allowing the adapter methods to hook into the right components.

## 4. Loading the Model and Initializing with the Interface

Now, let's load the Qwen 2.5 model and initialize it with our plugin interface.

In [ ]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B",  # Using the 1.5B version
    device_map="auto",  # Automatically distribute model across available GPUs
    torch_dtype="bfloat16",  # Use half-precision for faster computation
)

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")

# Set the pad token ID to be different from the model's EOS token
tokenizer.pad_token_id = 151645
model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
# Initialize the adapter framework with our plugin interface
# Remove the interface argument to use the default interface
adapters.init(model, interface=plugin_interface)

## 5. Adding and Training an Adapter

With the interface in place, we can now add an adapter to our model.
In this example, we'll train a [bottleneck adapter](https://docs.adapterhub.ml/methods.html#bottleneck-adapters). You can easily switch to [one of the other supported adapter methods](https://docs.adapterhub.ml/overview.html#table-of-adapter-methods) (e.g. LoRA) by changing the `adapter_config`.

In [ ]:
from adapters import SeqBnConfig, LoRAConfig

# Add a LoRA adapter
adapter_name = "qwen-math-adapter"
adapter_config = SeqBnConfig(
    reduction_factor=32,  # Bottleneck size
)
# Alternatively e.g.: 
# adapter_config = LoRAConfig(
#     r=32,  # Rank of the low-rank decomposition
#     alpha=16,  # Scaling factor for LoRA
# )

model.add_adapter(adapter_name, config=adapter_config)

# Activate the adapter
model.set_active_adapters(adapter_name)

# Set the model to train only the adapter parameters
model.train_adapter(adapter_name)

# Verify adapter was correctly added
print(model.adapter_summary())

## 6. Loading the GSM8K Dataset for Fine-tuning

For this example, we'll use the GSM8K dataset to fine-tune our model for solving grade school math problems.

In [ ]:
from datasets import load_dataset

# Load the GSM8K dataset
dataset = load_dataset("openai/gsm8k", "main")
print(dataset)

In [ ]:
# Explore sample data
print("Sample question:")
print(dataset["train"][0]["question"])
print("\nSample answer:")
print(dataset["train"][0]["answer"])

## 7. Preprocessing the Dataset

We need to tokenize our math problems and their solutions for training.

In [ ]:
def preprocess_function(examples):
    # Create full prompts with question and expected answer format
    prompts = [
        f"Solve the following math problem step-by-step:\n\nQuestion: {q}\n\nAnswer: {a} <|endoftext|>"
        for q, a in zip(examples["question"], examples["answer"])
    ]
    
    # Tokenize as regular sequences
    tokenized = tokenizer(prompts, padding="max_length", truncation=True, max_length=2048)
    
    # For causal language modeling, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["question", "answer"])

print("Dataset processed!")

## 8. Fine-tuning the Adapter

Now we can fine-tune our adapter for solving math problems.

In [ ]:
from transformers import TrainingArguments
import numpy as np


# Set up training arguments
training_args = TrainingArguments(
    output_dir="./qwen-math-adapter",
    per_device_train_batch_size=2,  # Increase or decrease based on GPU memory
    per_device_eval_batch_size=2,
    learning_rate=1e-4,
    num_train_epochs=1,  # More epochs for complex task
    save_steps=30,
    eval_steps=30,
    logging_steps=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="loss",  # Use loss as metric for best model
    greater_is_better=False,  # Lower loss is better
    push_to_hub=False,
    gradient_accumulation_steps=8,  # Accumulate gradients to simulate larger batch sizes
    bf16=True,  # Use mixed precision
    warmup_ratio=0.1,  # Add some warmup steps
)

In [ ]:
# Split dataset into train and validation
# Use a bugger/ smaller subset as needed
train_dataset = tokenized_dataset["train"].select(range(min(len(tokenized_dataset["train"]), 4000)))
eval_dataset = tokenized_dataset["test"].select(range(min(len(tokenized_dataset["test"]), 200)))

print(f"Training on {len(train_dataset)} examples and evaluating on {len(eval_dataset)} examples")

In [ ]:
from adapters import AdapterTrainer
from trl import DataCollatorForCompletionOnlyLM

# Initialize the trainer
trainer = AdapterTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    data_collator=DataCollatorForCompletionOnlyLM(response_template="Answer:", tokenizer=tokenizer),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train only the adapter parameters
trainer.train()

## 9. Saving and Loading the Adapter

After training, we can save just the adapter weights.

In [ ]:
# Save only the adapter weights
model.save_adapter("./qwen-math-adapter", adapter_name)

## 10. Testing the Adapter

Let's test our math problem-solving adapter on some new examples.

In [ ]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [ ]:
# Let's test the model with a few math problems
test_examples = [
    "Janet's ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four eggs. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
    "Carlos is planting a lemon tree. The tree will cost $90 to plant. Each year it will grow 7 lemons, which he can sell for $1.5 each. It costs $3 a year to water and feed the tree. How many years will it take before he starts earning money on the lemon tree?",
    "Two trains leave San Rafael at the same time. They begin traveling westward, both traveling for 80 miles. The next day, they travel northwards, covering 150 miles. What's the distance covered by each train in the two days?"
]

# Format the test examples with the prompt template
def to_prompt(text):
    return f"Solve the following math problem step-by-step:\n\nQuestion: {text}\n\nAnswer:"

for example in test_examples:
    print("=" * 50)
    print("Problem:")
    print(example)
    prompt = to_prompt(example)
    output = generator(prompt, max_new_tokens=500, do_sample=False, return_full_text=False)
    print("Solution:")
    print(output[0]["generated_text"])

## 11. Conclusion

In this notebook, we've demonstrated how to:

1. Create a plugin interface for adding adapter support to Qwen 2.5
2. Load and initialize the model with the adapter framework
3. Add a bottleneck adapter to the model
4. Fine-tune the adapter on the GSM8K math problem-solving task
5. Save and reload the adapter weights
6. Use the adapter for solving new math problems

The plugin interface approach allows you to use parameter-efficient fine-tuning with any Transformer model, even those not officially supported by the adapters library.